## Version Description
In this version, we deploy a Hybrid PINN to predict only one correction factor for the MTR ( `MTR-600` ) by the neural network.

In [1]:
from IPython.core.display import display, HTML,display_html
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
# Source: https://stackoverflow.com/questions/38783027/jupyter-notebook-display-two-pandas-tables-side-by-side
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

In [3]:
# import required libraries
import random
import pandas as pd
import numpy as np

from functools import reduce
import sys

#Set some numpy print options for displaying numpy arrays to fit maximum width of cell
np.set_printoptions(precision=3, edgeitems=30, linewidth=1000,formatter=dict(float=lambda x: "%.3g" % x))

# Disable Warnings for chained assignments Eg:Setting with Copy Warning
pd.options.mode.chained_assignment = None

from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show, output_file,output_notebook
output_notebook() # Set to output the plot in the notebook

Loading BokehJS ...

In [4]:
# import LTR and corresponding Restrictor dataframe
Restrictor_data = pd.read_csv('D:/OneDrive/Documents/Airbus/GitHub/ML_A350_Restrictors/TZ6/data/Zone6_RestrictorData_LTR.csv')
LTR_data = pd.read_csv('D:/OneDrive/Documents/Airbus/GitHub/ML_A350_Restrictors/TZ6/data/Zone6_LTRData.csv')
obs_HOVs = pd.read_csv('../data/LTR_dataset.csv', usecols = ['HoV'])[['HoV']]
LTR_data[['AMBP','MIXP']] *= 100 # Convert hPa to Pa
zero_MIXT_hoV_idx = LTR_data.index[LTR_data['MIXT'] == 20].tolist() # Points where originally, MIXT == 0 and substituted with 20° C
LTR_data[['MIXT','AMBT']] += 273.15 # Convert Celcius to Kelvin
print('Total LTR points:',len(LTR_data))

## Calculate Density = (0.5* Pmix + Pamb) / (R * Tmix) where R = 287 is the Gas constant in Ideal Gas Law
idx = LTR_data.columns.get_loc("AMBT")
LTR_data.insert(loc=(idx+1), column='Density(kg/m³)', value=np.round((0.5 * LTR_data['MIXP'] + LTR_data['AMBP'] ) / (LTR_data['MIXT']*287),4))
target_df = reduce(lambda left,right: pd.merge(left,right,left_on='HoV',right_on='HoV'), [LTR_data,Restrictor_data])
target_df['HoV'] = obs_HOVs.values
target_df.head()

Total LTR points: 34


,HoV,CAOLH_C66-C68,CAOLH_C68-C70,CAOLH_C70-C72,CAOLH_C72-C74,CAOLH_C74-C76,CAOLH_C76-C78,CAORH_C66-C68,CAORH_C68-C70,CAORH_C70-C72,...,R632_HS1,R633_HS1,R634_HS1,R635_HS1,R636_HS1,R637_HS1,R638_HS1,R639_HS1,R640_HS1,R641_HS1
0,A1,36.071000,38.802000,36.796000,36.794000,39.100000,38.318000,37.055000,39.596000,38.333000,...,41,41,41,41,41,41,41,41,41,41
1,A2,36.061000,38.697000,37.156000,36.738000,38.957000,38.326000,36.501000,40.797000,39.284000,...,41,41,41,41,41,41,41,41,41,41
2,A3,37.504234,40.911735,38.412733,37.859887,36.585486,38.504564,36.161971,37.068225,37.893405,...,41,41,41,41,41,41,41,41,41,41
3,A4,36.383413,39.101836,38.110486,37.597728,36.190671,37.706898,36.668551,37.759174,38.497755,...,41,41,41,41,41,41,41,41,41,41
4,A5,36.897000,39.925000,38.704000,38.175000,40.591000,39.913000,37.641000,38.309000,38.600000,...,41,41,41,41,41,41,41,41,41,41


In [5]:
target_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34 entries, 0 to 33
Data columns (total 58 columns):
HoV               34 non-null object
CAOLH_C66-C68     34 non-null float64
CAOLH_C68-C70     34 non-null float64
CAOLH_C70-C72     34 non-null float64
CAOLH_C72-C74     34 non-null float64
CAOLH_C74-C76     34 non-null float64
CAOLH_C76-C78     34 non-null float64
CAORH_C66-C68     34 non-null float64
CAORH_C68-C70     34 non-null float64
CAORH_C70-C72     34 non-null float64
CAORH_C72-C74     34 non-null float64
CAORH_C74-C76     34 non-null float64
CAORH_C76-C78     34 non-null float64
LAOLH_C66-C68     34 non-null float64
LAOLH_C68-C70     34 non-null float64
LAOLH_C70-C72     34 non-null float64
LAOLH_C72-C74     34 non-null float64
LAOLH_C74-C76     34 non-null float64
LAOLH_C76-C78     34 non-null float64
LAORH_C66-C68     34 non-null float64
LAORH_C68-C70     34 non-null float64
LAORH_C70-C72     34 non-null float64
LAORH_C72-C74     34 non-null float64
LAORH_C74-C76     34 non

In [6]:
MIXP = [1800, 2600, 3400]
# Define Restrictor Labels
MTR = 'R600_HD'
CLRS = ['R610_HS1','R611_HS1','R612_HS1','R613_HS1']
CAORS = ['R620_HS1','R620_HS2','R620_HS3','R621_HS1','R621_HS2','R621_HS3']
MHRS = CLRS + CAORS
LAORS = ['R630_HS1','R631_HS1','R632_HS1','R633_HS1','R634_HS1','R635_HS1','R636_HS1','R637_HS1','R638_HS1','R639_HS1','R640_HS1','R641_HS1']

In [7]:
# Duct Areas
MTR_DuctArea = 0.03464 # in sq.m
MHR_Duct_Areas = {'R610_HS1':0.01767,'R611_HS1':0.01767,'R612_HS1':0.02461,'R613_HS1':0.02461,
                 'R620_HS1':0.01090,'R620_HS2':0.00470,'R620_HS3':0.01085,'R621_HS1':0.01085,'R621_HS2':0.00470,'R621_HS3':0.01090}

## Define Initial Correction factor values from CFD
# Correction Factor for Duct Part `TZ6_zmix:
TZ6_zmix_cf = 1 # Start with initial c_f =1 

#3D Correction Factor for MTR
MTR_cf = 1.1664876228437


#3D Correction Factors for MultiHole Restrictors
MHR_cf = {'R610_HS1':0.740773983383846,'R611_HS1':0.740205245723713,'R612_HS1':0.83879652680329,'R613_HS1':0.81775586281569,
          'R620_HS1':0.757121843184036,'R620_HS2':0.773919779037984,'R620_HS3':0.789028593182055,
          'R621_HS1':0.714811824527744,'R621_HS2':0.866271631648556,'R621_HS3':0.822447592395462}

### Calculate Loss-Coefficient $\zeta$
**For Single Hole Restrictor:**<br>

    f0_f1 = A_Circular(Hole_Diameter) / Area_Overall
    l_cross = Thickness / Hole_Diameter
    Zeta_dash = 0.13 + 0.34 * 10 ^ -(3.4 * l_cross + 88.4 * l_cross ^ 2.3)
    Zeta_Single_Hole_Thick_Chamfered = ((1 - f0_f1 + (Zeta_dash ^ 0.5) * (1 - f0_f1) ^ 0.375) ^ 2) * f0_f1 ^ -2

In [8]:
def SHR_Zeta_3D(n_holes,hole_dia,MTR_DuctArea,cf):
    '''
    Computes the Zeta with 3D Correction Factor (cf) for Single Hole Retrictors
    '''
    MTR_New_Area = n_holes * (np.pi/4) * (hole_dia / 1000)**2 # Divide dia by 1000 to convert mm to m
    f0_f1 = MTR_New_Area/MTR_DuctArea
    l_cross = 1/hole_dia
    zeta_dash = 0.13 + 0.34 * 10**(-(3.4 * l_cross + 88.4 * l_cross**2.3))
    zeta_SHR_1D = ((1 - f0_f1 + (zeta_dash**0.5) * (1 - f0_f1)**0.375)**2) * f0_f1**(-2) # 1D Zeta
    zeta_SHR_3D = zeta_SHR_1D * cf # Zeta with 3D Correction Factor    
    return MTR_New_Area,zeta_SHR_3D

In [9]:
target_df[MTR+'_Area'],target_df[MTR+'_Zeta3D'] = zip(*[SHR_Zeta_3D(1,dia,MTR_DuctArea,MTR_cf) for dia in target_df[MTR]])
target_df[[MTR,'R600_HD_Area','R600_HD_Zeta3D']].head()

,R600_HD,R600_HD_Area,R600_HD_Zeta3D
0,148,0.017203,4.949375
1,149,0.017437,4.730821
2,152,0.018146,4.127180
3,154,0.018627,3.764504
4,148,0.017203,4.949375


**For Multi Hole Restrictors:**<br>

    Area_Free = Number_of_Holes * A_Circular(Hole_Diameter)
    f0_f1 = Area_Free / Area_Overall
    l_cross = Thickness / Hole_Diameter
    phi = 0.25 + (0.535 * l_cross ^ 8) / (0.05 + l_cross ^ 7)
    tau = (2.4 - l_cross) * 10 ^ (-phi)
    Zeta_Multi_Hole = (0.5 * (1 - f0_f1) ^ 0.75 + tau * (1 - f0_f1) ^ 1.375 + (1 - f0_f1) ^ 2 + 0.02 * l_cross) / f0_f1 ^ 2

In [10]:
def MHR_Zeta_3D(nr_holes,hole_dia,MHR_DuctArea,cf):
    '''
    Computes the Zeta with 3D Correction Factor (cf) for Multi Hole Restrictors
    '''
    MHR_New_Area = nr_holes * (np.pi/4) * (hole_dia / 1000)**2 # Divide dia by 1000 to convert mm to m
    f0_f1 = MHR_New_Area/MHR_DuctArea
    l_cross = (0.00144*1000)/hole_dia
    phi = 0.25 + (0.535 * l_cross**8) / (0.05 + l_cross**7)
    tau = (2.4 - l_cross) * 10**(-phi)
    zeta_MHR_1D = (0.5 * (1 - f0_f1)**0.75 + tau * (1 - f0_f1)**1.375 + (1 - f0_f1)**2 + 0.02 * l_cross) / f0_f1**2 # 1D Zeta     
    zeta_MHR_3D = zeta_MHR_1D * cf # Zeta with 3D Correction Factor    
    return MHR_New_Area,zeta_MHR_3D

In [11]:
# Calculate Zeta for Multi-Hole Restrictors
for clr in CLRS:
    if (clr == 'R610_HS1') | (clr == 'R611_HS1'):
        hole_dia = 8
    else:
        hole_dia = 10
    MHR_nr_holes = target_df[clr].values
    target_df[clr+'_Area'], target_df[clr+'_Zeta3D'] = zip(*[MHR_Zeta_3D(ele,hole_dia,MHR_Duct_Areas[clr],MHR_cf[clr]) for ele in MHR_nr_holes])
    
for caor in CAORS:
    CAOR_nr_holes = target_df[caor].values
    target_df[caor+'_Area'],target_df[caor+'_Zeta3D'] = zip(*[MHR_Zeta_3D(ele,8,MHR_Duct_Areas[caor],MHR_cf[caor]) for ele in CAOR_nr_holes])

In [12]:
# LTR Dataframe col index for Restrictor Zeta and Area values
target_df.iloc[:,58:]

,R600_HD_Area,R600_HD_Zeta3D,R610_HS1_Area,R610_HS1_Zeta3D,R611_HS1_Area,R611_HS1_Zeta3D,R612_HS1_Area,R612_HS1_Zeta3D,R613_HS1_Area,R613_HS1_Zeta3D,...,R620_HS2_Area,R620_HS2_Zeta3D,R620_HS3_Area,R620_HS3_Zeta3D,R621_HS1_Area,R621_HS1_Zeta3D,R621_HS2_Area,R621_HS2_Zeta3D,R621_HS3_Area,R621_HS3_Zeta3D
0,0.017203,4.949375,0.006585,7.506119,0.006836,6.741078,0.009425,7.920333,0.009425,7.721657,...,0.00191,6.107864,0.00568,2.786638,0.003770,8.801675,0.00191,6.836716,0.00568,2.95119
1,0.017437,4.730821,0.006585,7.506119,0.006836,6.741078,0.009425,7.920333,0.009425,7.721657,...,0.00191,6.107864,0.00568,2.786638,0.003770,8.801675,0.00191,6.836716,0.00568,2.95119
2,0.018146,4.127180,0.006585,7.506119,0.006836,6.741078,0.008954,9.156844,0.009425,7.721657,...,0.00191,6.107864,0.00568,2.786638,0.003770,8.801675,0.00191,6.836716,0.00568,2.95119
3,0.018627,3.764504,0.006585,7.506119,0.006836,6.741078,0.009425,7.920333,0.009425,7.721657,...,0.00191,6.107864,0.00568,2.786638,0.003770,8.801675,0.00191,6.836716,0.00568,2.95119
4,0.017203,4.949375,0.007288,5.595443,0.007691,4.756868,0.010210,6.271347,0.010210,6.114035,...,0.00191,6.107864,0.00568,2.786638,0.003770,8.801675,0.00191,6.836716,0.00568,2.95119
5,0.017203,4.949375,0.006585,7.506119,0.006836,6.741078,0.009425,7.920333,0.009425,7.721657,...,0.00191,6.107864,0.00568,2.786638,0.003770,8.801675,0.00191,6.836716,0.00568,2.95119
6,0.017203,4.949375,0.006585,7.506119,0.006836,6.741078,0.009425,7.920333,0.009425,7.721657,...,0.00191,6.107864,0.00568,2.786638,0.003770,8.801675,0.00191,6.836716,0.00568,2.95119
7,0.018385,3.942093,0.006585,7.506119,0.007037,6.198696,0.009425,7.920333,0.009817,6.862828,...,0.00191,6.107864,0.00568,2.786638,0.003770,8.801675,0.00191,6.836716,0.00568,2.95119
8,0.018146,4.127180,0.006585,7.506119,0.006836,6.741078,0.009425,7.920333,0.009817,6.862828,...,0.00191,6.107864,0.00568,2.786638,0.003770,8.801675,0.00191,6.836716,0.00568,2.95119
9,0.017671,4.521189,0.006585,7.506119,0.006836,6.741078,0.009425,7.920333,0.009425,7.721657,...,0.00191,6.107864,0.00568,2.786638,0.003770,8.801675,0.00191,6.836716,0.00568,2.95119


**Store Loss Coefficient Values and Names for all Restrictors**:

In [13]:
# Gather col_names and their corresponding column indices
zeta_col_names = [col for col in target_df.columns if 'Zeta3D' in col]
zeta_col_names_idx = [target_df.columns.get_loc(col) for col in zeta_col_names if col in target_df]
area_col_names = [col for col in target_df.columns if 'Area' in col]
area_col_names_idx = [target_df.columns.get_loc(col) for col in area_col_names if col in target_df]
pt_var_idx = [target_df.columns.get_loc(col) for col in ['MIXP','AMBP','AMBT'] if col in target_df]
req_flow_rates_idx = [target_df.columns.get_loc(col) for col in ['TZ6_Flow'] if col in target_df]
# print(zeta_col_names_idx)
print(zeta_col_names)

['R600_HD_Zeta3D', 'R610_HS1_Zeta3D', 'R611_HS1_Zeta3D', 'R612_HS1_Zeta3D', 'R613_HS1_Zeta3D', 'R620_HS1_Zeta3D', 'R620_HS2_Zeta3D', 'R620_HS3_Zeta3D', 'R621_HS1_Zeta3D', 'R621_HS2_Zeta3D', 'R621_HS3_Zeta3D']


In [14]:
LTR_zeta_df_col_idx = sorted([0] + req_flow_rates_idx + pt_var_idx + zeta_col_names_idx)
target_df.iloc[:,LTR_zeta_df_col_idx].to_csv('data_output/LTR_Zeta_df.csv', index=False)

In [15]:
LTR_zeta_df = target_df[['HoV','MIXP','AMBP','AMBT','TZ6_Flow']+zeta_col_names]
# LTR_zeta_df.tail()

In [16]:
final_df = target_df[['HoV','MIXP','Density(kg/m³)','R600_HD','R600_HD_Zeta3D','TZ6_Flow']]
final_df.reset_index(drop=True,inplace = True)

### Training data Preparation

In [17]:
from Hybrid_PINN_v1_1 import NormbyMax, DeNormbyMax, PINN

In [18]:
input_features = ['MIXP','Density(kg/m³)','R600_HD_Zeta3D','TZ6_Flow']
features_max, df_rescaled = NormbyMax(final_df,input_features)
print('Max value of all features:',features_max)

V_max_org = features_max['TZ6_Flow']
print('Maximum TZ6 Flow:',V_max_org)

Max value of all features: {'MIXP': 2632.9689, 'Density(kg/m³)': 1.238, 'R600_HD_Zeta3D': 5.4148802645163, 'TZ6_Flow': 764.0123719999998}
Maximum TZ6 Flow: 764.0123719999998


In [19]:
# Display original and Scaled DataFrame Side-by-side
# print( '{:<60s} {:<20s}'.format('Original Input Dataframe:','Rescaled Input Dataframe:') )
# display_side_by_side(final_df[['HoV']+input_features],df_rescaled)

In [20]:
df_rescaled_inv = DeNormbyMax(df_rescaled,features_max,input_features)

In [21]:
train_data = df_rescaled
# train_data = df_rescaled.iloc[[7,15]]
print("Number of training samples:", len(train_data),'\nSCALED FEATURES:')
train_data

Number of training samples: 34 
SCALED FEATURES:


,MIXP,Density(kg/m³),R600_HD_Zeta3D,TZ6_Flow
0,0.987478,0.986026,0.914032,0.966062
1,0.987478,0.987722,0.873670,0.965658
2,0.989830,0.976575,0.762192,0.969853
3,0.987440,0.999031,0.695215,0.958246
4,0.987478,1.000000,0.914032,0.970963
5,0.987478,0.985299,0.914032,0.973195
6,0.987478,0.978837,0.914032,0.968664
7,0.988183,0.967367,0.728011,0.971982
8,0.985737,0.960420,0.762192,0.968232
9,0.987478,0.996446,0.834956,0.968331


In [22]:
# Store indices and values of train data
train_data_idx = train_data.index.values.tolist()
train_R600HD_series = final_df['R600_HD'].iloc[train_data_idx]
X_train = train_data.values

### Predict $c_f$ with Physics Informed Neural Network

In [23]:
np.random.seed(42)
# Define custom MSE function
MSE = lambda y_hat,y_true: np.mean((np.asarray(y_hat)-np.asarray(y_true))**2)

**Train the Neural Network**:

In [24]:
### Set the hyperparameters here ###
iterations = 100
learning_rate = 0.01 ### Important Parameter
hidden_nodes = 2 ### Important Parameter
output_nodes = 1
MTR_epsi = MTR_cf * 0.01

In [25]:
%%time
N_i = X_train.shape[1]
network = PINN(N_i, hidden_nodes, output_nodes, learning_rate, MTR_epsi)

MSE_flowloss_hist = []
err_loss_hist = []
losses_hist = {'HoV':[],'MSE_flowloss':[],'Error_loss':[]}
c_f_hist = {'HoV':[],'c_f_hat':[],'Early_Stopping_Reason':[],'FlowRate_Diff_(LTR-FDDN)':[]}

for idx in train_data_idx:    
    ## Stochastic Gradient Descent with mini-batch training
    print('Old_Zeta:',target_df[MTR+'_Zeta3D'].loc[idx])
    for ii in range(iterations):    
        # Go through one record at a time from the training data set
        X_train = train_data[input_features].loc[[idx]].values
        dia_train = np.atleast_1d(train_R600HD_series.loc[idx])
        error,V_hat_FDDN,V_true_LTR,break_flag,c_f = network.train(X_train,dia_train,[idx],MTR_DuctArea,target_df,zeta_col_names,V_max_org)        
        # Printing out the training progress
        MSE_flowloss = MSE(V_hat_FDDN,V_true_LTR) # MSE between FDDN predicted flow rate and true flow rate on UN-scaled original values        
        sys.stdout.write("\rProgress: {:2.1f}".format(100 * ii/float(iterations))+ "% ... MSE Flowloss: " + str(round(MSE_flowloss,4))[:5] + " ... Error Function loss: " + str(round(error,4)))
        print('... Training Iteration:',ii)    
        sys.stdout.flush()
        MSE_flowloss_hist.append(MSE_flowloss)
        err_loss_hist.append(error)
        early_stopping_bf = network.early_stopping(err_loss_hist, 10) # Call early stopping function to monitor last 10 epochs of the loss array        
        if (1 in break_flag) or (early_stopping_bf == 1) or (ii == (iterations - 1)): # early stopping
            HoV = target_df[['HoV']].loc[idx].values.tolist()     
            print('EARLY STOPPING ACTIVATED - Terminating Neural Network Training')                        
            print('MSEFlow Loss History for {}:'.format(HoV[0]),MSE_flowloss_hist)
            print('Error Loss History for {}:'.format(HoV[0]),err_loss_hist)
            losses_hist['HoV'].append(HoV[0])            
            losses_hist['MSE_flowloss'].append(MSE_flowloss_hist)
            losses_hist['Error_loss'].append(err_loss_hist)
            c_f_hist['HoV'].append(HoV[0])            
            c_f_hist['c_f_hat'].append(c_f[0])
            c_f_hist['FlowRate_Diff_(LTR-FDDN)'].append(V_true_LTR[0] - V_hat_FDDN[0] )                        
            if (1 in break_flag):
                c_f_hist['Early_Stopping_Reason'].append('FlowratesConverged')
            elif (early_stopping_bf == 1):
                c_f_hist['Early_Stopping_Reason'].append('ErrorLossStagned')
            err_loss_hist = []
            MSE_flowloss_hist = []
            break

Old_Zeta: 4.949375283472444

NN output is -ve. Taking |(c_f)| value: [0.0421]
New_Zeta: 0.17853191165055501
FDDN Solver Output: [[99.8 47.6 150 98.4 52 160 15.7 33.9 33.9 33.9 33.2 32.8 15.3 33.4 33.7 33.1 32.7 32.7]]
New_Zeta_epsi: -0.22802566448527942
FDDN Solver Output: [[99.3 47.4 150 98 51.7 160 15.7 33.8 33.7 33.8 33.1 32.6 15.3 33.3 33.5 33 32.6 32.6]]
Row ID: 0
HoV: ['A1']
Flow Rate Difference (LTR - FDDN): -234.95168481669975 l/s
No. of Records: 1
Progress: 0.0% ... MSE Flowloss: 55202 ... Error Function loss: 0.5061... Training Iteration: 0

NN output (C_f): 3.7015545449499645
New_Zeta: 15.705595341456295
FDDN Solver Output: [[51.5 24.6 77.6 50.8 26.8 82.7 8.13 17.6 17.5 17.6 17.2 17 7.94 17.3 17.4 17.1 16.9 16.9]]
New_Zeta_epsi: 15.75508909429102
FDDN Solver Output: [[51.4 24.5 77.5 50.7 26.8 82.6 8.12 17.5 17.5 17.5 17.2 16.9 7.93 17.3 17.4 17.1 16.9 16.9]]
Row ID: 0
HoV: ['A1']
Flow Rate Difference (LTR - FDDN): 235.48324481503278 l/s
No. of Records: 1
Progress: 1.0% ... M

FDDN Solver Output: [[71.4 34.1 108 70.7 37.3 115 11.2 24.3 24.2 24.3 23.8 23.4 11 23.8 24 23.7 23.4 23.4]]
Row ID: 4
HoV: ['A5']
Flow Rate Difference (LTR - FDDN): 43.46094519391943 l/s
No. of Records: 1
Progress: 0.0% ... MSE Flowloss: 1888. ... Error Function loss: 0.0431... Training Iteration: 0

NN output (C_f): 0.9019803423683578
New_Zeta: 3.8270780806168356
FDDN Solver Output: [[77.8 37.1 117 77 40.7 126 12.2 26.4 26.4 26.4 25.9 25.5 11.9 26 26.2 25.8 25.4 25.5]]
New_Zeta_epsi: 3.87657183345156
FDDN Solver Output: [[77.6 37 117 76.8 40.6 125 12.2 26.4 26.3 26.4 25.8 25.5 11.9 25.9 26.1 25.7 25.4 25.4]]
Row ID: 4
HoV: ['A5']
Flow Rate Difference (LTR - FDDN): -17.39036269653991 l/s
No. of Records: 1
Progress: 1.0% ... MSE Flowloss: 302.4 ... Error Function loss: 0.0051... Training Iteration: 1

NN output (C_f): 1.056602813892973
New_Zeta: 4.483137025303893
FDDN Solver Output: [[75.1 35.9 113 74.4 39.3 121 11.8 25.5 25.5 25.5 25 24.6 11.5 25.1 25.3 24.9 24.6 24.6]]
New_Zeta_epsi: 

FDDN Solver Output: [[76.7 36.6 116 74.3 39.3 121 11.9 25.7 25.6 25.7 25.1 24.8 11.6 25.3 25.5 25.1 24.8 24.8]]
New_Zeta_epsi: 4.4415501667192245
FDDN Solver Output: [[76.5 36.5 115 74.2 39.2 121 11.9 25.6 25.6 25.6 25.1 24.7 11.6 25.2 25.4 25 24.7 24.7]]
Row ID: 8
HoV: ['C3']
Flow Rate Difference (LTR - FDDN): 0.4846403809057165 l/s
Percentage Diff between flowrates 0.0655363236952848 %
No. of Records: 1
Progress: 0.0% ... MSE Flowloss: 0.234 ... Error Function loss: 0.0297... Training Iteration: 0
EARLY STOPPING ACTIVATED - Terminating Neural Network Training
MSEFlow Loss History for C3: [0.23487629880443797]
Error Loss History for C3: [0.02968890030502196]
Old_Zeta: 4.521188627716888

NN output (C_f): 1.255599167673778
New_Zeta: 4.8665760070547455
FDDN Solver Output: [[72.8 34.7 110 71.8 37.9 117 11.5 24.8 24.8 24.8 24.3 24 11.2 24.4 24.6 24.2 23.9 23.9]]
New_Zeta_epsi: 4.911787893331915
FDDN Solver Output: [[72.7 34.7 110 71.7 37.9 117 11.5 24.8 24.7 24.8 24.2 23.9 11.2 24.4 24.6 2

FDDN Solver Output: [[78 36.7 113 77.7 40 120 11.8 25.5 25.4 25.5 24.9 24.6 11.5 25.1 25.3 24.8 24.5 24.6]]
Row ID: 13
HoV: ['D2']
Flow Rate Difference (LTR - FDDN): -2.3693536949252803 l/s
No. of Records: 1
Progress: 1.0% ... MSE Flowloss: 5.613 ... Error Function loss: 0.0002... Training Iteration: 1

NN output (C_f): 0.9989784049619979
New_Zeta: 4.238638224200118
FDDN Solver Output: [[77.9 36.6 112 77.6 39.9 120 11.8 25.4 25.4 25.4 24.9 24.6 11.5 25 25.2 24.8 24.5 24.5]]
New_Zeta_epsi: 4.288131977034843
FDDN Solver Output: [[77.7 36.5 112 77.4 39.8 119 11.7 25.4 25.3 25.4 24.8 24.5 11.5 24.9 25.1 24.7 24.4 24.4]]
Row ID: 13
HoV: ['D2']
Flow Rate Difference (LTR - FDDN): 0.9282665420299736 l/s
Percentage Diff between flowrates 0.1259020104383007 %
No. of Records: 1
Progress: 2.0% ... MSE Flowloss: 0.861 ... Error Function loss: 0.0... Training Iteration: 2
EARLY STOPPING ACTIVATED - Terminating Neural Network Training
MSEFlow Loss History for D2: [47.764625894060345, 5.61383693165607

FDDN Solver Output: [[68.9 32.9 104 67.9 35.9 111 10.9 23.5 23.4 23.5 23 22.7 10.6 23.1 23.3 22.9 22.6 22.6]]
Row ID: 15
HoV: ['E2']
Flow Rate Difference (LTR - FDDN): -2.8993620634939816 l/s
No. of Records: 1
Progress: 11.0% ... MSE Flowloss: 8.406 ... Error Function loss: 0.1054... Training Iteration: 11

NN output (C_f): 1.4591175866574453
New_Zeta: 6.190996267476019
FDDN Solver Output: [[69 32.9 104 68.1 36 111 10.9 23.5 23.5 23.5 23 22.7 10.6 23.2 23.3 23 22.7 22.7]]
New_Zeta_epsi: 6.240490020310744
FDDN Solver Output: [[68.9 32.9 104 67.9 35.9 111 10.9 23.5 23.4 23.5 23 22.7 10.6 23.1 23.3 22.9 22.6 22.6]]
Row ID: 15
HoV: ['E2']
Flow Rate Difference (LTR - FDDN): -2.9076447586610357 l/s
No. of Records: 1
Progress: 12.0% ... MSE Flowloss: 8.454 ... Error Function loss: 0.1054... Training Iteration: 12
NO CHANGE in TRAIN LOSS for the Last 10 Epochs
EARLY STOPPING ACTIVATED - Terminating Neural Network Training
MSEFlow Loss History for E2: [5079.760264347995, 329.8664708973091, 24.6

FDDN Solver Output: [[78.1 37.3 118 77 40.7 126 12.4 26.8 26.8 26.8 26.2 25.9 12.4 26.9 27.1 26.7 26.4 26.4]]
New_Zeta_epsi: 3.69834541596977
FDDN Solver Output: [[77.9 37.2 117 76.8 40.6 125 12.4 26.7 26.7 26.8 26.2 25.8 12.3 26.9 27.1 26.6 26.3 26.3]]
Row ID: 20
HoV: ['I1']
Flow Rate Difference (LTR - FDDN): -40.14500341084863 l/s
No. of Records: 1
Progress: 0.0% ... MSE Flowloss: 1611. ... Error Function loss: 0.0019... Training Iteration: 0

NN output (C_f): 1.3568902839811718
New_Zeta: 4.800849107973673
FDDN Solver Output: [[73.6 35.1 111 72.6 38.3 118 11.7 25.3 25.2 25.3 24.7 24.4 11.7 25.4 25.6 25.2 24.9 24.9]]
New_Zeta_epsi: 4.842120912552894
FDDN Solver Output: [[73.4 35 111 72.4 38.3 118 11.7 25.2 25.2 25.2 24.7 24.4 11.6 25.3 25.5 25.1 24.8 24.8]]
Row ID: 20
HoV: ['I1']
Flow Rate Difference (LTR - FDDN): 4.0985001176052265 l/s
No. of Records: 1
Progress: 1.0% ... MSE Flowloss: 16.79 ... Error Function loss: 0.0637... Training Iteration: 1

NN output (C_f): 1.3059396155756557

FDDN Solver Output: [[74.6 35.6 112 73.2 38.7 119 11.8 25.6 25.5 25.6 25 24.7 11.6 25.2 25.4 25 24.7 24.7]]
Row ID: 25
HoV: ['Q1']
Flow Rate Difference (LTR - FDDN): -5.420774155504091 l/s
No. of Records: 1
Progress: 1.0% ... MSE Flowloss: 29.38 ... Error Function loss: 0.0006... Training Iteration: 1

NN output (C_f): 1.0109037764637236
New_Zeta: 4.289237251400235
FDDN Solver Output: [[74.1 35.4 112 72.7 38.4 119 11.8 25.4 25.4 25.4 24.9 24.5 11.5 25 25.2 24.8 24.5 24.5]]
New_Zeta_epsi: 4.33873100423496
FDDN Solver Output: [[73.9 35.3 111 72.6 38.3 118 11.7 25.3 25.3 25.4 24.8 24.5 11.5 24.9 25.1 24.7 24.4 24.4]]
Row ID: 25
HoV: ['Q1']
Flow Rate Difference (LTR - FDDN): 1.3085545792717994 l/s
Percentage Diff between flowrates 0.18064658403696088 %
No. of Records: 1
Progress: 2.0% ... MSE Flowloss: 1.712 ... Error Function loss: 0.0001... Training Iteration: 2
EARLY STOPPING ACTIVATED - Terminating Neural Network Training
MSEFlow Loss History for Q1: [725.0971155445876, 29.384792444981


NN output (C_f): 1.082484338558038
New_Zeta: 4.1956003521402225
FDDN Solver Output: [[76.3 36.4 115 74 39.1 121 11.8 25.5 25.5 25.6 25 24.7 11.6 25.1 25.4 24.9 24.6 24.6]]
New_Zeta_epsi: 4.240812238417392
FDDN Solver Output: [[76.1 36.3 115 73.8 39 120 11.8 25.5 25.4 25.5 24.9 24.6 11.5 25.1 25.3 24.9 24.6 24.6]]
Row ID: 30
HoV: ['S4']
Flow Rate Difference (LTR - FDDN): -1.1686128192567367 l/s
Percentage Diff between flowrates 0.15898378497125432 %
No. of Records: 1
Progress: 2.0% ... MSE Flowloss: 1.365 ... Error Function loss: 0.0034... Training Iteration: 2
EARLY STOPPING ACTIVATED - Terminating Neural Network Training
MSEFlow Loss History for S4: [180.43786455173722, 6.925203757514847, 1.3656559213311783]
Error Loss History for S4: [0.0001560842275548213, 0.005869434396458228, 0.0034030028507066948]
Old_Zeta: 2.8397674398682766

NN output (C_f): 1.1392413037117133
New_Zeta: 2.7734373662249294
FDDN Solver Output: [[82.6 39.4 125 80.1 42.3 131 13 28.1 28.1 28.1 27.5 27.2 12.5 27.2 2

FDDN Solver Output: [[72.3 34.5 109 71.3 37.7 116 11.4 24.6 24.6 24.7 24.1 23.8 11.2 24.3 24.5 24.1 23.8 23.8]]
New_Zeta_epsi: 5.364363874739358
FDDN Solver Output: [[72.2 34.4 109 71.2 37.6 116 11.4 24.6 24.5 24.6 24.1 23.8 11.1 24.2 24.4 24 23.7 23.7]]
Row ID: 32
HoV: ['T1']
Flow Rate Difference (LTR - FDDN): 16.62546746065391 l/s
No. of Records: 1
Progress: 0.0% ... MSE Flowloss: 276.4 ... Error Function loss: 0.0321... Training Iteration: 0

NN output (C_f): 1.0993881088957065
New_Zeta: 4.664673869275173
FDDN Solver Output: [[74.6 35.6 113 73.6 38.9 120 11.8 25.4 25.4 25.4 24.9 24.6 11.5 25 25.2 24.8 24.5 24.5]]
New_Zeta_epsi: 4.714167622109898
FDDN Solver Output: [[74.5 35.5 112 73.4 38.8 120 11.7 25.4 25.3 25.4 24.8 24.5 11.5 25 25.2 24.8 24.5 24.5]]
Row ID: 32
HoV: ['T1']
Flow Rate Difference (LTR - FDDN): -5.64724689327943 l/s
No. of Records: 1
Progress: 1.0% ... MSE Flowloss: 31.89 ... Error Function loss: 0.005... Training Iteration: 1

NN output (C_f): 1.1437709567483663
New

In [26]:
cf_df = pd.DataFrame(data=c_f_hist)
losses = pd.DataFrame(data=losses_hist)

In [27]:
mseflow_last_loss = []
error_last_loss = []
for i in range(len(losses)):
    loss1 = losses.MSE_flowloss[i][-1]
    loss2 = losses.Error_loss[i][-1]
    mseflow_last_loss.append(loss1)
    error_last_loss.append(loss2)

cf_df['Final_MSELoss'] = mseflow_last_loss
cf_df['Final_ErrorLoss'] = error_last_loss

In [28]:
cf_df['MaxEpochs'] = None # Initialize Empty Column for storing epochs trained

for i in range(len(losses['HoV'])):
    nr_epochs_trained = len(losses['MSE_flowloss'][i])
    cf_df['MaxEpochs'].loc[i] = nr_epochs_trained
    
cf_df.to_csv('data_output/NN_1CF_Output_SingleHoVLoop.csv',index=False)
cf_df

,HoV,c_f_hat,Early_Stopping_Reason,FlowRate_Diff_(LTR-FDDN),Final_MSELoss,Final_ErrorLoss,MaxEpochs
0,A1,1.029904,FlowratesConverged,0.960946,0.923417,0.000448,7
1,A2,1.071943,FlowratesConverged,1.613860,2.604543,0.002590,2
2,A3,1.156327,FlowratesConverged,0.856708,0.733949,0.012220,2
3,A4,1.287343,FlowratesConverged,-1.296258,1.680284,0.041284,2
4,A5,1.015784,FlowratesConverged,2.141023,4.583979,0.000128,5
5,A6,0.952170,FlowratesConverged,1.315715,1.731105,0.001145,3
6,C1,0.980190,FlowratesConverged,-1.293032,1.671931,0.000198,3
7,C2,1.248436,FlowratesConverged,-1.888206,3.565320,0.030863,3
8,C3,1.243675,FlowratesConverged,0.484640,0.234876,0.029689,1
9,C4,1.058977,FlowratesConverged,1.366617,1.867643,0.001741,3


In [29]:
print('Max no. of epochs for:',cf_df['HoV'].iloc[[cf_df.MaxEpochs.idxmax()]].values)
cf_df.iloc[[cf_df.MaxEpochs.idxmax()]]

Max no. of epochs for: ['E2']


,HoV,c_f_hat,Early_Stopping_Reason,FlowRate_Diff_(LTR-FDDN),Final_MSELoss,Final_ErrorLoss,MaxEpochs
15,E2,1.459118,ErrorLossStagned,-2.907645,8.454398,0.105402,13


**Plot Training Loss**

In [30]:
from bokeh.palettes import Category20,Colorblind,Spectral,Set2,YlGnBu,RdPu
temp_list = []
bokeh_palettes = [Colorblind,YlGnBu,RdPu,Set2]
for palette in bokeh_palettes:
    for key in palette.keys():
        temp_list.append(palette[key])
    
color_palette = [y for x in temp_list for y in x]

In [31]:
num_lines = len(losses.HoV) # no. of lines to draw
colors = color_palette[0:num_lines]
labels = losses.HoV.values.tolist()

p1 = figure(width=1600)
for i in range(num_lines):
    x = list(range(1,len(losses['MSE_flowloss'][i])+1))
    y1 = losses['MSE_flowloss'][i]    
    p1.line(x, y1, line_width=2, color=colors[i], alpha=0.8, legend='MSE Flow Loss for {}'.format(labels[i]))    
p1.yaxis.axis_label = "MSE FlowRate Loss"
p1.xaxis.axis_label = "Epochs"
p1.legend.click_policy="hide"
show(p1)

In [32]:
p2 = figure(width=1600)
for i in range(num_lines):
    x = list(range(1,len(losses['MSE_flowloss'][i])+1))     
    y2 = losses['Error_loss'][i]    
    err_loss_lineplot = p2.line(x, y2, line_width=2, color= colors[i], alpha=0.5, legend='Error Loss for {}'.format(labels[i]))
p2.yaxis.axis_label = "Error Loss"
p2.xaxis.axis_label = "Epochs"
p2.legend.click_policy="hide"
show(p2)